In [97]:
from genespectrakg.adapters.process_eggnog_data_input import *
from genespectrakg.adapters.genespectra_adapter_wilcox_logit import *
from genespectrakg.adapters.process_genespectra_input import *
import pandas as pd

import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [10]:
eggnog = read_eggnog_raw("../data/eggnogv5_9443_members.tsv", eggnog_dataset_name='Primates', eggnog_version='v5')

In [11]:
eggnog
# this is from emapper, understand what does the seed_ortholog mean and write
# code to substract the gene-to-OG mapping from the emapper output

,eggnog_dataset_id,eggnog_og_id,num_sequences,num_species,sequences_id,species_id,eggnog_dataset_name
0,9443,4M5J0,16,15,"27679.XP_003930946.1,30611.ENSOGAP00000008143,...","27679,30611,60711,61622,61853,9478,9483,9541,9...",Primates
1,9443,4M5J1,16,15,"27679.XP_003924097.1,30611.ENSOGAP00000009583,...","27679,30611,60711,61622,61853,9478,9483,9541,9...",Primates
2,9443,4M5J2,24,15,"27679.XP_010331361.1,27679.XP_010331362.1,2767...","27679,30611,60711,61622,61853,9478,9483,9541,9...",Primates
3,9443,4M5J3,13,13,"27679.XP_003935201.2,30611.ENSOGAP00000021337,...","27679,30611,60711,61622,9478,9541,9544,9555,95...",Primates
4,9443,4M5J4,16,15,"27679.XP_003930470.1,30611.ENSOGAP00000003659,...","27679,30611,60711,61622,61853,9478,9483,9541,9...",Primates
...,...,...,...,...,...,...,...
23672,9443,4MS18,9,9,"60711.ENSCSAP00000007651,61622.XP_010365614.1,...","60711,61622,9541,9544,9555,9593,9597,9598,9606",Primates
23673,9443,4MS19,5,5,"9544.ENSMMUP00000004036,9597.XP_003813236.1,95...","9544,9597,9598,9601,9606",Primates
23674,9443,4MS1A,2,1,"9555.ENSPANP00000016036,9555.ENSPANP00000017532",9555,Primates
23675,9443,4MS1B,5,5,"30611.ENSOGAP00000016940,61853.ENSNLEP00000015...","30611,61853,9544,9593,9598",Primates


In [12]:
species_names_ids = {
    'hsapiens': '9606', 
    "mmulatta": '9544',
    'cjacchus': '9483', # marmoset
    'ptroglodytes': '9598', # chimpanzee
    'ggorilla': '9593', # western gorilla # seems to be 9595 in ncbi
}

In [13]:
eggnog_filtered = filter_eggnog_ogs(eggnog, species_ids_keep=list(species_names_ids.values()))

In [14]:
eggnog_filtered

,eggnog_dataset_id,eggnog_og_id,num_sequences,num_species,eggnog_dataset_name,sequence_id_filtered,species_id,sequence_id
0,9443,4M5J0,16,15,Primates,9483.ENSCJAP00000018717,9483,ENSCJAP00000018717
1,9443,4M5J0,16,15,Primates,9544.ENSMMUP00000008535,9544,ENSMMUP00000008535
2,9443,4M5J0,16,15,Primates,9593.ENSGGOP00000010276,9593,ENSGGOP00000010276
3,9443,4M5J0,16,15,Primates,9598.ENSPTRP00000025451,9598,ENSPTRP00000025451
4,9443,4M5J0,16,15,Primates,9606.ENSP00000310241,9606,ENSP00000310241
...,...,...,...,...,...,...,...,...
97222,9443,4MS19,5,5,Primates,9606.ENSP00000382828,9606,ENSP00000382828
97223,9443,4MS1B,5,5,Primates,9544.ENSMMUP00000021351,9544,ENSMMUP00000021351
97224,9443,4MS1B,5,5,Primates,9593.ENSGGOP00000024696,9593,ENSGGOP00000024696
97225,9443,4MS1B,5,5,Primates,9598.ENSPTRP00000014262,9598,ENSPTRP00000014262


In [15]:
eggnog_filtered.to_csv("../data/eggnogv5_9443_members_MTG_sps_only.tsv", index=False)

In [ ]:
# the former functionalities is when we don't need gene annotations from emapper
# but now that e want to do GNN, we need to use the actual emapper results
# this is how to process it, ref. platy analysis

In [24]:
eggnog = pd.read_csv("../data/wilcox/chimp_emapper_annot_skim.csv")

eggnog = eggnog.assign(ogs=eggnog.eggNOG_OGs.str.split(",")).explode("ogs")
eggnog[['og_name', 'eggnog_dataset_name']] = eggnog['ogs'].str.split('|', n=1, expand=True)
eggnog[['og_id', 'eggnog_dataset_id']] = eggnog['og_name'].str.split('@', n=1, expand=True)
eggnog.rename(columns={'query': 'peptide'}, inplace=True)


In [25]:
eggnog.head()

,peptide,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,BiGG_Reaction,PFAMs,gene,renamed_gene,is_a_GO_tf,ogs,og_name,eggnog_dataset_name,og_id,eggnog_dataset_id
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,-,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,KOG3865@1|root,KOG3865@1,root,KOG3865,1
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,-,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,KOG3865@2759|Eukaryota,KOG3865@2759,Eukaryota,KOG3865,2759
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,-,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,38CBU@33154|Opisthokonta,38CBU@33154,Opisthokonta,38CBU,33154
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,-,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,3BA04@33208|Metazoa,3BA04@33208,Metazoa,3BA04,33208
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,-,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,3CRUG@33213|Bilateria,3CRUG@33213,Bilateria,3CRUG,33213


In [26]:
eggnog['species_of_origin'] = 'P.troglodytes'

In [27]:
eggnog.gene

0        ENSPTRP00000071240.1
0        ENSPTRP00000071240.1
0        ENSPTRP00000071240.1
0        ENSPTRP00000071240.1
0        ENSPTRP00000071240.1
                 ...         
26026    ENSPTRP00000068323.1
26026    ENSPTRP00000068323.1
26026    ENSPTRP00000068323.1
26026    ENSPTRP00000068323.1
26027      ## Rate: 95.81 q/s
Name: gene, Length: 258154, dtype: object

In [30]:
eggnog=eggnog.loc[~eggnog.gene.str.contains('|'.join(['queries scanned', '## Rate:', '## Total time (seconds):']) ), ]


/var/folders/37/wf962dk574750g0xnnlxjwvm0000gp/T/ipykernel_89767/4172331321.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  eggnog=eggnog.loc[~eggnog.gene.str.contains('|'.join(['queries scanned', '## Rate:', '## Total time (seconds):']) ), ]


In [38]:
eggnog.loc[eggnog.gene.str.contains('|'.join(['queries scanned', '## Rate:', '## Total time (seconds):']) ), ]

/var/folders/37/wf962dk574750g0xnnlxjwvm0000gp/T/ipykernel_89767/2199508385.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  eggnog.loc[eggnog.gene.str.contains('|'.join(['queries scanned', '## Rate:', '## Total time (seconds):']) ), ]


,peptide,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,PFAMs,gene,renamed_gene,is_a_GO_tf,ogs,og_name,eggnog_dataset_name,og_id,eggnog_dataset_id,species_of_origin


In [31]:
eggnog.replace('3\'','3 prime', regex=True,  inplace=True)
eggnog.replace('2\'','2 prime', regex=True,  inplace=True)
eggnog.replace('5\'','5 prime', regex=True,  inplace=True)
eggnog.replace('\'','prime', regex=True,  inplace=True)


In [32]:
eggnog.head()

,peptide,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,PFAMs,gene,renamed_gene,is_a_GO_tf,ogs,og_name,eggnog_dataset_name,og_id,eggnog_dataset_id,species_of_origin
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,KOG3865@1|root,KOG3865@1,root,KOG3865,1,P.troglodytes
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,KOG3865@2759|Eukaryota,KOG3865@2759,Eukaryota,KOG3865,2759,P.troglodytes
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,38CBU@33154|Opisthokonta,38CBU@33154,Opisthokonta,38CBU,33154,P.troglodytes
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,3BA04@33208|Metazoa,3BA04@33208,Metazoa,3BA04,33208,P.troglodytes
0,ENSPTRP00000071240.1,10029.XP_007625217.1,5.570000e-67,214.0,"KOG3865@1|root,KOG3865@2759|Eukaryota,38CBU@33...",33208|Metazoa,K,the binding appears to require additional rece...,ARRB2,"GO:0000003,GO:0000822,GO:0001541,GO:0001664,GO...",...,"Arrestin_C,Arrestin_N",ENSPTRP00000071240.1,ENSPTRP00000071240.1 | ARRB2 | the binding app...,False,3CRUG@33213|Bilateria,3CRUG@33213,Bilateria,3CRUG,33213,P.troglodytes


In [33]:
eggnog.to_csv("../data/wilcox_chimp_annot_skim_OGs_processed.csv", index=False)

In [39]:
def process_emapper_results(emapper_file_path, species_name, species_scientific_name):
    eggnog = pd.read_csv(emapper_file_path)

    eggnog = eggnog.assign(ogs=eggnog.eggNOG_OGs.str.split(",")).explode("ogs")
    eggnog[['og_name', 'eggnog_dataset_name']] = eggnog['ogs'].str.split('|', n=1, expand=True)
    eggnog[['og_id', 'eggnog_dataset_id']] = eggnog['og_name'].str.split('@', n=1, expand=True)
    eggnog.rename(columns={'query': 'peptide'}, inplace=True)

    eggnog['species_of_origin'] = species_scientific_name

    eggnog=eggnog.loc[~eggnog.gene.str.contains('|'.join(['queries scanned', '## Rate:', '## Total time \(seconds\):']) ), ]

    eggnog.replace('3\'','3 prime', regex=True,  inplace=True)
    eggnog.replace('2\'','2 prime', regex=True,  inplace=True)
    eggnog.replace('5\'','5 prime', regex=True,  inplace=True)
    eggnog.replace('\'','prime', regex=True,  inplace=True)

    eggnog.to_csv(f"../data/wilcox_{species_name}_annot_skim_OGs_processed.csv", index=False)

In [40]:
process_emapper_results("../data/wilcox/gorilla_emapper_annot_skim.csv", 'gorilla', 'G.gorilla')

In [41]:
process_emapper_results("../data/wilcox/human_emapper_annot_skim.csv", 'human', 'H.sapiens')
process_emapper_results("../data/wilcox/macaque_emapper_annot_skim.csv", 'macaque', 'M.mulatta')
process_emapper_results("../data/wilcox/marmoset_emapper_annot_skim.csv", 'marmoset', 'C.jacchus')
process_emapper_results("../data/wilcox/chimp_emapper_annot_skim.csv", 'chimp', 'P.troglodytes')

/var/folders/37/wf962dk574750g0xnnlxjwvm0000gp/T/ipykernel_89767/3400072745.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  eggnog = pd.read_csv(emapper_file_path)


In [42]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# process wilcox marker genes table

In [93]:
def process_wilcox_scanpy(wilcox_scanpy_file_path, species_name, species_scientific_name):

    mks = pd.read_csv(wilcox_scanpy_file_path,   index_col=0)
    mks.rename(columns={'group': 'cell_type', 'names': 'preferred_name'}, inplace=True)
    mks['species_of_origin'] = species_scientific_name
    mks['logfc_rank'] = mks.groupby('cell_type')['logfoldchanges'].rank( ascending=False)
    mks['pvals_adj_rank'] = mks.groupby('cell_type')['pvals_adj'].rank( ascending=True)
    mks.sort_values(['cell_type', 'logfc_rank'], inplace=True)
    mks.reset_index(drop=True, inplace=True)
    mks.to_csv(f"../data/wilcox/{species_name}_wilcox_marker_genes_processed.csv", index=False)

    name_diff = pd.read_csv(f"/Users/ysong/SOFTWARE/GeneSpectraKG/data/NCBI/{species_name}_ncbi_name_diff_ensembl_name.csv", index_col=0)
    merged = mks.merge(name_diff,  left_on='preferred_name', right_on='ncbi_gene_name', how='left')
    merged['ensembl_gene_name_use'] = np.select([~merged['external_gene_name'].isna(), 
                                             merged['external_gene_name'].isna()],
                                            [merged['external_gene_name'], 
                                             merged['preferred_name']],
                                             default=None)
    merged.drop(["external_gene_name", "ncbi_gene_name", "ensembl_gene_id"], axis=1, inplace=True)
    merged.to_csv(f"../data/wilcox/{species_name}_wilcox_marker_genes_processed_ensembl_name_use.csv", index=False)
    


In [94]:
process_wilcox_scanpy("../data/wilcox/chimp_cellxgene_obs_all_wilcox.csv", "chimp", 'P.troglodytes')

In [95]:
process_wilcox_scanpy("../data/wilcox/human_cellxgene_obs_all_wilcox.csv", "human", 'H.sapiens')

In [96]:
process_wilcox_scanpy("../data/wilcox/macaque_cellxgene_obs_all_wilcox.csv", 'macaque', 'M.mulatta')
process_wilcox_scanpy("../data/wilcox/marmoset_cellxgene_obs_all_wilcox.csv", 'marmoset', 'C.jacchus')
process_wilcox_scanpy("../data/wilcox/gorilla_cellxgene_obs_all_wilcox.csv", 'gorilla', 'G.gorilla')


In [105]:
len(merged.preferred_name.unique())

16416

In [106]:
len(mks.preferred_name.unique())

16416

In [107]:
merged

,cell_type,preferred_name,scores,logfoldchanges,pvals,pvals_adj,species_of_origin,logfc_rank,pvals_adj_rank,ensembl_gene_name_use
0,Astro,GLI3,133.356050,12.324874,0.000000e+00,0.000000e+00,C.jacchus,1.0,333.0,GLI3
1,Astro,CCDC129,126.831450,11.977973,0.000000e+00,0.000000e+00,C.jacchus,2.0,333.0,CCDC129
2,Astro,LOC108592491,105.616040,11.251725,0.000000e+00,0.000000e+00,C.jacchus,3.0,333.0,LOC108592491
3,Astro,NWD1,122.063150,11.179513,0.000000e+00,0.000000e+00,C.jacchus,4.0,333.0,NWD1
4,Astro,LOC108587679,122.962210,11.146167,0.000000e+00,0.000000e+00,C.jacchus,5.0,333.0,LOC108587679
...,...,...,...,...,...,...,...,...,...,...
79784,Vip,SNAP91,-3.818193,0.005247,1.344329e-04,6.314507e-04,C.jacchus,1828.0,1597.0,SNAP91
79785,Vip,LOC108588832,3.680941,0.004155,2.323752e-04,1.056378e-03,C.jacchus,1829.0,1633.0,LOC108588832
79786,Vip,SIPA1L1,-19.500273,0.003730,1.091992e-84,3.684249e-83,C.jacchus,1830.0,378.0,SIPA1L1
79787,Vip,MADD,4.054191,0.002820,5.030814e-05,2.488436e-04,C.jacchus,1831.0,1557.0,MADD


In [108]:
mks

,cell_type,preferred_name,scores,logfoldchanges,pvals,pvals_adj,species_of_origin,logfc_rank,pvals_adj_rank
0,Astro,GLI3,133.356050,12.324874,0.000000e+00,0.000000e+00,C.jacchus,1.0,333.0
1,Astro,CCDC129,126.831450,11.977973,0.000000e+00,0.000000e+00,C.jacchus,2.0,333.0
2,Astro,LOC108592491,105.616040,11.251725,0.000000e+00,0.000000e+00,C.jacchus,3.0,333.0
3,Astro,NWD1,122.063150,11.179513,0.000000e+00,0.000000e+00,C.jacchus,4.0,333.0
4,Astro,LOC108587679,122.962210,11.146167,0.000000e+00,0.000000e+00,C.jacchus,5.0,333.0
...,...,...,...,...,...,...,...,...,...
79780,Vip,SNAP91,-3.818193,0.005247,1.344329e-04,6.314507e-04,C.jacchus,1828.0,1597.0
79781,Vip,LOC108588832,3.680941,0.004155,2.323752e-04,1.056378e-03,C.jacchus,1829.0,1633.0
79782,Vip,SIPA1L1,-19.500273,0.003730,1.091992e-84,3.684249e-83,C.jacchus,1830.0,378.0
79783,Vip,MADD,4.054191,0.002820,5.030814e-05,2.488436e-04,C.jacchus,1831.0,1557.0


In [132]:
name_diff.loc[name_diff.duplicated('ncbi_gene_name') | name_diff.duplicated('ncbi_gene_name', keep='last'), :]

,external_gene_name,ncbi_gene_name,ensembl_gene_id
773,NKX6-2,NaN,ENSCJAG00000010137
1409,KRTAP11-1,NaN,ENSCJAG00000022606
3754,CFH,LOC100399888,ENSCJAG00000008415
3755,CFHR5,LOC100399888,ENSCJAG00000037347
5990,HLA-DPB1,NaN,ENSCJAG00000021575
6656,SEC22A,LOC100411007,ENSCJAG00000007704
6657,PDIA5,LOC100411007,ENSCJAG00000007712
6719,NKX6-1,NaN,ENSCJAG00000005335
7050,HLA-DMA,NaN,ENSCJAG00000021558
7955,KRTAP29-1,NaN,ENSCJAG00000022475


In [ ]:
# the 4 more rows in mks comes from duplicated ncbi_gene_name
# is this actually just ok for the KG? 

In [123]:
name_check = name_diff.loc[name_diff.duplicated('external_gene_name') | name_diff.duplicated('external_gene_name', keep='last'), :]['ncbi_gene_name'].values

In [ ]:
# why the merged as more rows than mks?

In [138]:
mks.loc[mks['preferred_name']=='NaN', :]

,cell_type,preferred_name,scores,logfoldchanges,pvals,pvals_adj,species_of_origin,logfc_rank,pvals_adj_rank


In [137]:
merged.loc[merged['preferred_name']=='NaN', :]

,cell_type,preferred_name,scores,logfoldchanges,pvals,pvals_adj,species_of_origin,logfc_rank,pvals_adj_rank,ensembl_gene_name_use


In [ ]:
# TODO: figure out where the duplication come from that caused merged to have more rows than mks
# TODO: then re-write the get nodes and get edges